In [9]:
filename = 'pairs_with_similarity_merged_from_server.parquet'
# filename = 'pairs_with_similarity/out_4.parquet'


# get number of rows with parquetfile
import pyarrow.parquet as pq
parquet_file = pq.ParquetFile(filename)
print('number of rows: ' + str(parquet_file.metadata.num_rows))
print(parquet_file.metadata.schema)

number of rows: 93529599
required group field_id=-1 schema {
  optional binary field_id=-1 census_id (String);
  optional binary field_id=-1 deepdao_organization_id (String);
  optional binary field_id=-1 deepdao_name (String);
  optional binary field_id=-1 census_name (String);
  optional binary field_id=-1 row_id (String);
  optional float field_id=-1 similarity;
}



In [12]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [13]:
con.execute(f'select count(*) from {filename}').df()

,count_star()
0,93529599


In [20]:
# count distinct rows
con.execute(f'select count(distinct row_id) from {filename}').df()

OutOfMemoryException: Out of Memory Error: failed to allocate data of size 2.0MB (6.8GB/6.8GB used)

In [15]:
con.execute(f'select * from {filename} limit 5').df()

,census_id,deepdao_organization_id,deepdao_name,census_name,row_id,similarity
0,30f80d86-7669-50de-83ec-ec1343a2ea54,61e9839b-f517-4141-a379-12b1f4e9085c,Tough Turtles Tribe,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_61e9839b-...,0.178658
1,30f80d86-7669-50de-83ec-ec1343a2ea54,015b39b2-98fe-4183-a4c9-1b9aec49b1fd,toweb3,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_015b39b2-...,0.208167
2,30f80d86-7669-50de-83ec-ec1343a2ea54,4fce2184-7330-411b-ad69-34bdfe5aa481,TPT DAO,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_4fce2184-...,0.272460
3,30f80d86-7669-50de-83ec-ec1343a2ea54,f533f01e-a196-4cb7-9303-50528c7317a2,Trips,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_f533f01e-...,0.159215
4,30f80d86-7669-50de-83ec-ec1343a2ea54,6107a8fa-d2e3-45ea-8619-1ab59203e093,UkraineDAO,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_6107a8fa-...,0.197024


In [23]:
query = f"""
SELECT
    count(distinct row_id)
FROM
    {filename}
WHERE
    similarity > 0.85
"""
con.execute(query).df()

,count(DISTINCT row_id)
0,2955


In [26]:
query = f"""
SELECT
    count(distinct row_id)
FROM
    {filename}
WHERE
    similarity == 1
"""
con.execute(query).df()

,count(DISTINCT row_id)
0,311


In [ ]:
# there are 2955 potential matches of deepdao names < - > census names that
# we should review manually